In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
true_df = pd.read_csv("True.csv")
fake_df = pd.read_csv("Fake.csv")
true_df['label'] = 1 
fake_df['label'] = 0 
df = pd.concat([true_df, fake_df])
df = df.sample(frac=1).reset_index(drop=True) 
print(df.head())


                                               title  \
0  Episode #172 – SUNDAY WIRE: ‘Trumpe Le Monde’ ...   
1  TRUMP’S NEW COMM DIRECTOR Puts CNN’s Chris Cuo...   
2  Clamor for justice: Yugoslav court leaves glob...   
3  YOU’VE BEEN “TRUMPED”: Trump’s Response To Pre...   
4  DID OBAMA COMMIT A FELONY When He Sent $400 Mi...   

                                                text          subject  \
0   Episode #172 of SUNDAY WIRE SHOW resumes this...      Middle-east   
1  Remember when the media used to think they con...         politics   
2  BELGRADE (Reuters) - When a court on the Dutch...        worldnews   
3  You know what they say about playing with fire...         politics   
4  Fox News Senior Judicial Analyst Judge Andrew ...  Government News   

                 date  label  
0    February 5, 2017      0  
1        Jul 26, 2017      0  
2  November 20, 2017       1  
3        Sep 11, 2015      0  
4         Aug 6, 2016      0  


In [2]:
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
  
    text = text.lower()
    
 
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    

    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    
 
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    

    tokens = [lemmatizer.lemmatize(word) for word in tokens]
 
    cleaned_text = ' '.join(tokens)
    
    return cleaned_text

df['cleaned_text'] = df['text'].apply(clean_text)

print(df[['text', 'cleaned_text']].head(2))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arrkr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\arrkr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\arrkr\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


                                                text  \
0   Episode #172 of SUNDAY WIRE SHOW resumes this...   
1  Remember when the media used to think they con...   

                                        cleaned_text  
0  episode sunday wire show resume february host ...  
1  remember medium used think controlled narrativ...  


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=8000)  

X = tfidf.fit_transform(df['cleaned_text']).toarray()

y = df['label'].values

print("TF-IDF matrix shape:", X.shape)


TF-IDF matrix shape: (44898, 8000)


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


In [7]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()


model.fit(X_train, y_train)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [8]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9863028953229399

Confusion Matrix:
 [[4608   73]
 [  50 4249]]

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.98      0.99      4681
           1       0.98      0.99      0.99      4299

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [9]:
import joblib

joblib.dump(model, 'model/fake_news_model.pkl')


joblib.dump(tfidf, 'model/tfidf_vectorizer.pkl')


['model/tfidf_vectorizer.pkl']

In [10]:
import os

if not os.path.exists("model"):
    os.makedirs("model")


In [11]:

joblib.dump(model, 'model/fake_news_model.pkl')


joblib.dump(tfidf, 'model/tfidf_vectorizer.pkl')


['model/tfidf_vectorizer.pkl']

In [12]:
model = joblib.load('model/fake_news_model.pkl')
tfidf = joblib.load('model/tfidf_vectorizer.pkl')


In [13]:
predict_news("Breaking: Scientists find cure for common cold")


NameError: name 'predict_news' is not defined

In [44]:

import joblib
model = joblib.load('model/fake_news_model.pkl')
tfidf = joblib.load('model/tfidf_vectorizer.pkl')

def clean_text(text):
    import re
    import string
    from nltk.corpus import stopwords
    from nltk.stem import WordNetLemmatizer

    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()

    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

def predict_news(text):
    cleaned = clean_text(text)
    vector = tfidf.transform([cleaned]).toarray()
    prediction = model.predict(vector)
    return "REAL" if prediction[0] == 1 else "FAKE"

# Try it
print(predict_news("India wins the T20 World Cup in dramatic final!"))


FAKE


In [47]:
predict_news("Kenya and South Africa have also threatened to quit the ICC.")


'FAKE'

In [48]:
print(df['label'].value_counts())


label
0    23481
1    21417
Name: count, dtype: int64


In [49]:
real_article = df[df['label'] == 1]['text'].iloc[0]  # 1 = REAL
print("Sample Real News:\n", real_article)
print("Prediction:", predict_news(real_article))


Sample Real News:
 BELGRADE (Reuters) - When a court on the Dutch North Sea coast issues its final verdict this week, it will signal the end of an experiment that has reverberated around the world, from the killing fields of Rwanda to the CIA s secret cells in Europe. The International Criminal Tribunal for the former Yugoslavia (ICTY), set up by the United Nations in 1993, marked the biggest leap in the field of international criminal law since the Allies tried the Nazis in Nuremberg. Created in answer to the worst war crimes in Europe since World War Two, it set a precedent of accountability that has since put the Khmer Rouge and Liberia s Charles Taylor in the dock and paved the way for a court with global ambition. Almost 25 years later, its legacy is under threat. The International Criminal Court (ICC), opened in 2002, is undermined by renewed West-Russia rivalry, stone-walling and revolt in Africa, barrel bombs in Syria and a boycott by three of five permanent members of the U.N.

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X = df['text']     
y = df['label']    
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

tfidf = TfidfVectorizer(stop_words='english', max_df=0.7)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

model = LogisticRegression(max_iter=1000, class_weight='balanced')
model.fit(X_train_tfidf, y_train)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [58]:
print(df['label'].value_counts())


label
0    23481
1    21417
Name: count, dtype: int64


In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
X_train_tfidf = vectorizer.fit_transform(X_train)   
X_test_tfidf = vectorizer.transform(X_test)         


In [60]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000, class_weight='balanced')
model.fit(X_train_tfidf, y_train)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [61]:
print(model.predict(X_test_tfidf[:10]))


[1 0 1 1 1 1 1 1 0 0]


In [62]:
text = "The ICTY issued 161 indictments and secured 83 convictions over its 24 years of operation."
cleaned = clean_text(text)
vector = vectorizer.transform([cleaned])

proba = model.predict_proba(vector)
print("Probabilities [Fake, Real]:", proba)
print("Prediction:", model.predict(vector))


Probabilities [Fake, Real]: [[0.81011085 0.18988915]]
Prediction: [0]


In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7, ngram_range=(1,2))


X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000, class_weight='balanced')
model.fit(X_train_tfidf, y_train)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [64]:
cleaned = clean_text("The ICTY issued 161 indictments and secured 83 convictions over its 24 years of operation.")
vector = vectorizer.transform([cleaned])
print("Probabilities [Fake, Real]:", model.predict_proba(vector))
print("Prediction:", model.predict(vector))


Probabilities [Fake, Real]: [[0.72991658 0.27008342]]
Prediction: [0]


In [65]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_train_tfidf, y_train, cv=5, scoring='f1')
print("Cross-validated F1 scores:", scores)
print("Mean F1:", scores.mean())


Cross-validated F1 scores: [0.98015757 0.98109916 0.98280886 0.98175981 0.98163801]
Mean F1: 0.9814926812787357


In [66]:
text = """The ICTY issued 161 indictments and secured 83 convictions over its 24 years of operation, 
according to the United Nations tribunal's final report released in The Hague."""
print("Prediction:", predict_news(text))


ValueError: X has 110954 features, but LogisticRegression is expecting 3340280 features as input.

In [67]:
def predict_news(text):
    cleaned = clean_text(text)
    vector = vectorizer.transform([cleaned])   # use the same fitted vectorizer
    prediction = model.predict(vector)
    return "REAL" if prediction[0] == 1 else "FAKE"


In [68]:
import joblib
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")
joblib.dump(model, "logreg_model.pkl")


['logreg_model.pkl']

In [69]:
vectorizer = joblib.load("tfidf_vectorizer.pkl")
model = joblib.load("logreg_model.pkl")


In [70]:
import joblib

# After training
joblib.dump(model, "logreg_model.pkl")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")


['tfidf_vectorizer.pkl']

In [71]:
import joblib
model = joblib.load("logreg_model.pkl")
vectorizer = joblib.load("tfidf_vectorizer.pkl")

def predict_news(text):
    cleaned = clean_text(text)
    vector = vectorizer.transform([cleaned])   
    prediction = model.predict(vector)
    return "REAL" if prediction[0] == 1 else "FAKE"


In [72]:
import joblib

model = joblib.load("logreg_model.pkl")
vectorizer = joblib.load("tfidf_vectorizer.pkl")

def predict_news(text):
    cleaned = clean_text(text)
    vector = vectorizer.transform([cleaned])   
    prediction = model.predict(vector)
    proba = model.predict_proba(vector)
    return prediction[0], proba


In [84]:
text = "As U.S. budget fight looms, Republicans flip...."
print("Prediction:", predict_news(text))


Prediction: (np.int64(0), array([[0.72260754, 0.27739246]]))


In [74]:
text = """The ICTY issued 161 indictments and secured 83 convictions over its 24 years of operation, 
according to the United Nations tribunal's final report released in The Hague."""

label, proba = predict_news(text)
print("Prediction:", "REAL" if label == 1 else "FAKE")
print("Probabilities [Fake, Real]:", proba)


Prediction: FAKE
Probabilities [Fake, Real]: [[0.67150974 0.32849026]]


In [81]:
import pandas as pd

df = pd.read_csv("true.csv")   
print(df.head())


                                               title  \
0  As U.S. budget fight looms, Republicans flip t...   
1  U.S. military to accept transgender recruits o...   
2  Senior U.S. Republican senator: 'Let Mr. Muell...   
3  FBI Russia probe helped by Australian diplomat...   
4  Trump wants Postal Service to charge 'much mor...   

                                                text       subject  \
0  WASHINGTON (Reuters) - The head of a conservat...  politicsNews   
1  WASHINGTON (Reuters) - Transgender people will...  politicsNews   
2  WASHINGTON (Reuters) - The special counsel inv...  politicsNews   
3  WASHINGTON (Reuters) - Trump campaign adviser ...  politicsNews   
4  SEATTLE/WASHINGTON (Reuters) - President Donal...  politicsNews   

                 date  
0  December 31, 2017   
1  December 29, 2017   
2  December 31, 2017   
3  December 30, 2017   
4  December 29, 2017   


In [83]:

df['prediction'] = df['text'].apply(predict_news)
real_news = df[df['prediction'] == 'REAL']
print(real_news[['title', 'text']].head())


Empty DataFrame
Columns: [title, text]
Index: []


In [85]:
print(df.columns)


Index(['title', 'text', 'subject', 'date', 'prediction'], dtype='object')


In [1]:
real_news = df[df['prediction'] == 1]
print(real_news[['title', 'text']].head(10))


NameError: name 'df' is not defined

In [2]:
print(df['prediction'].value_counts())


NameError: name 'df' is not defined